# part 1, 2, 3

In [1]:
!pip install -q kaggle

!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

# Step 4: Download the dataset
!kaggle datasets download -d pkdarabi/cardetection


 81% 78.0M/96.6M [00:00<00:00, 216MB/s]
100% 96.6M/96.6M [00:00<00:00, 207MB/s]


In [ ]:
!unzip cardetection.zip

The traffic sign image dataset comprises 4969 samples meticulously divided into three parts—Train, Valid, and Test. Featuring distinct classes such as Green Light, Red Light, Speed Limits ranging from 10 to 120, and Stop signs, the dataset presents a valuable resource for machine learning applications. With applications in autonomous vehicle navigation, the model derived from this dataset can empower self-driving cars to accurately recognize and respond to various traffic signs, ensuring adherence to traffic regulations. Additionally, the dataset finds utility in driver assistance systems, enhancing road safety by monitoring and alerting drivers who may violate speed limits or ignore red lights. Furthermore, it can serve as a foundation for road safety training programs, enabling simulations for new drivers to better understand and respond to different traffic scenarios. Beyond individual vehicles, the dataset can contribute to smart city initiatives, allowing authorities to employ the model in connected infrastructure for real-time monitoring of traffic compliance.

In [3]:
import os
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from sklearn.model_selection import StratifiedKFold

# Define the directories
directories = ['/content/train', '/content/valid', '/content/test']

# Define the model
def create_model():
    model = Sequential()
    model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(64, 64, 3)))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Flatten())
    model.add(Dense(units=128, activation='relu'))
    model.add(Dense(units=2, activation='softmax'))
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# Prepare the image data generator
datagen = ImageDataGenerator(rescale=1./255)

# K-fold Cross Validation model evaluation
fold_no = 1
for directory in directories[:-1]:  # Exclude the test directory
    print(f'Training for fold {fold_no} ...')

    train_data = datagen.flow_from_directory(
        directory,
        target_size=(64, 64),
        batch_size=32,
        class_mode='categorical'
    )

    # Create a new model for each fold
    model = create_model()

    # Fit data to model
    model.fit(train_data, epochs=10, verbose=1)

    fold_no = fold_no + 1

# Validation on the 'valid' set
valid_directory = '/content/valid'
valid_data = datagen.flow_from_directory(
    valid_directory,
    target_size=(64, 64),
    batch_size=32,
    class_mode='categorical'
)

# Create a new model for validation
model = create_model()

# Evaluate the model on the validation set
model.evaluate(valid_data, verbose=1)

# Testing on the 'test' set
test_directory = '/content/test'
test_data = datagen.flow_from_directory(
    test_directory,
    target_size=(64, 64),
    batch_size=32,
    class_mode='categorical'
)

# Create a new model for testing
model = create_model()

# Evaluate the model on the test set
model.evaluate(test_data, verbose=1)


Training for fold 1 ...
Found 3530 images belonging to 2 classes.
Epoch 1/10
111/111 [==============================] - 10s 87ms/step - loss: 0.0053 - accuracy: 0.9997
Epoch 2/10
111/111 [==============================] - 9s 84ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 3/10
111/111 [==============================] - 10s 91ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 4/10
111/111 [==============================] - 10s 86ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 5/10
111/111 [==============================] - 10s 86ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 6/10
111/111 [==============================] - 10s 87ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 7/10
111/111 [==============================] - 10s 85ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 8/10
111/111 [==============================] - 10s 88ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 9/10
111/111 [==============================] - 9s 84ms/step - loss: 0.0000e+00 - acc

[0.7796540856361389, 0.015673981979489326]

This code is for training, validating, and testing a Convolutional Neural Network (CNN) model using Keras and Scikit-learn libraries. The CNN model is defined with a sequential architecture consisting of a convolutional layer, a max pooling layer, a flattening layer, and two dense layers. The model is trained using images from the ‘train’ and ‘valid’ directories, and then evaluated on the ‘valid’ and ‘test’ directories. The images are preprocessed using Keras’s ImageDataGenerator to rescale the pixel values. The model is trained and evaluated separately for each directory in a form of K-fold Cross Validation, where K is the number of directories. The model’s performance is evaluated based on its accuracy on the validation and test sets. The code is designed to print the training progress and the evaluation results.

In [5]:
import os
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from sklearn.model_selection import StratifiedKFold

# Define the directories
directories = ['/content/train', '/content/valid', '/content/test']

# Define the model
def create_model(kernel_size, conv_stride, pool_size, pool_stride):
    model = Sequential()
    model.add(Conv2D(32, kernel_size, strides=conv_stride, activation='relu', input_shape=(64, 64, 3)))
    model.add(MaxPooling2D(pool_size=pool_size, strides=pool_stride))
    model.add(Flatten())
    model.add(Dense(units=128, activation='relu'))
    model.add(Dense(units=2, activation='softmax'))
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# Prepare the image data generator
datagen = ImageDataGenerator(rescale=1./255)

# Define configurations
configs = [((3, 3), (1, 1), (2, 2), (2, 2)), ((5, 5), (2, 2), (3, 3), (2, 2))]

# Variables to store the best configuration and its accuracy
best_acc = 0
best_config = None

# K-fold Cross Validation model evaluation
fold_no = 1
for config in configs:
    kernel_size, conv_stride, pool_size, pool_stride = config
    print(f'Testing configuration: {config}')

    for directory in directories[:-1]:  # Exclude the test directory
        print(f'Training for fold {fold_no} ...')

        train_data = datagen.flow_from_directory(
            directory,
            target_size=(64, 64),
            batch_size=32,
            class_mode='categorical'
        )

        # Create a new model for each fold
        model = create_model(kernel_size, conv_stride, pool_size, pool_stride)

        # Fit data to model
        history = model.fit(train_data, epochs=3, verbose=1)

        # Check if this configuration is the best
        if max(history.history['accuracy']) > best_acc:
            best_acc = max(history.history['accuracy'])
            best_config = config
            # Save the best model
            model.save('best_model.h5')

        fold_no = fold_no + 1

print(f'Best configuration: {best_config}')


Testing configuration: ((3, 3), (1, 1), (2, 2), (2, 2))
Training for fold 1 ...
Found 3530 images belonging to 2 classes.
Epoch 1/3
111/111 [==============================] - 10s 88ms/step - loss: 0.0063 - accuracy: 0.9949
Epoch 2/3
111/111 [==============================] - 10s 87ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 3/3
111/111 [==============================] - 10s 88ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Training for fold 2 ...
Found 801 images belonging to 2 classes.
Epoch 1/3


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


26/26 [==============================] - 3s 76ms/step - loss: 0.0272 - accuracy: 0.9850
Epoch 2/3
26/26 [==============================] - 2s 76ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 3/3
26/26 [==============================] - 2s 93ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Testing configuration: ((5, 5), (2, 2), (3, 3), (2, 2))
Training for fold 3 ...
Found 3530 images belonging to 2 classes.
Epoch 1/3
111/111 [==============================] - 5s 41ms/step - loss: 0.0064 - accuracy: 0.9975
Epoch 2/3
111/111 [==============================] - 5s 43ms/step - loss: 3.5353e-07 - accuracy: 1.0000
Epoch 3/3
111/111 [==============================] - 5s 46ms/step - loss: 2.7842e-07 - accuracy: 1.0000
Training for fold 4 ...
Found 801 images belonging to 2 classes.
Epoch 1/3
26/26 [==============================] - 2s 55ms/step - loss: 0.0385 - accuracy: 0.9600
Epoch 2/3
26/26 [==============================] - 1s 40ms/step - loss: 7.5451e-07 - accuracy: 1.0000
Epoch 3/3
26/

This code is designed to test different configurations of a Convolutional Neural Network (CNN) model for image classification. It uses Keras and Scikit-learn libraries to create the CNN model with varying kernel size and stride for convolutional layers, and varying pooling size and stride for pooling layers. The model is trained and evaluated on images from different directories using K-fold Cross Validation. The code prints the configuration being tested, and at the end, it prints the best configuration based on accuracy and saves the corresponding model. The configurations are defined in a list, and the best configuration is determined by comparing the accuracy of the current configuration with the highest accuracy found so far.

# part 4

In [7]:
import os
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from sklearn.model_selection import StratifiedKFold

# Define the directories
directories = ['/content/train', '/content/valid', '/content/test']

# Define the model
def create_model(kernel_size, conv_stride, pool_size, pool_stride):
    model = Sequential()
    model.add(Conv2D(32, kernel_size, strides=conv_stride, activation='relu', input_shape=(64, 64, 3)))
    model.add(MaxPooling2D(pool_size=pool_size, strides=pool_stride))
    model.add(Flatten())
    model.add(Dense(units=128, activation='relu'))
    model.add(Dense(units=2, activation='softmax'))
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# Prepare the image data generator with data augmentation
datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Define configurations
configs = [((3, 3), (1, 1), (2, 2), (2, 2)), ((5, 5), (2, 2), (3, 3), (2, 2))]

# Variables to store the best configuration and its accuracy
best_acc = 0
best_config = None

# K-fold Cross Validation model evaluation
fold_no = 1
for config in configs:
    kernel_size, conv_stride, pool_size, pool_stride = config
    print(f'Testing configuration: {config}')

    for directory in directories[:-1]:  # Exclude the test directory
        print(f'Training for fold {fold_no} ...')

        train_data = datagen.flow_from_directory(
            directory,
            target_size=(64, 64),
            batch_size=32,
            class_mode='categorical'
        )

        # Create a new model for each fold
        model = create_model(kernel_size, conv_stride, pool_size, pool_stride)

        # Fit data to model
        history = model.fit(train_data, epochs=3, verbose=1)

        # Check if this configuration is the best
        if max(history.history['accuracy']) > best_acc:
            best_acc = max(history.history['accuracy'])
            best_config = config
            # Save the best model
            model.save('best_model.h5')

        fold_no = fold_no + 1

print(f'Best configuration: {best_config}')


Testing configuration: ((3, 3), (1, 1), (2, 2), (2, 2))
Training for fold 1 ...
Found 3530 images belonging to 2 classes.
Epoch 1/3
111/111 [==============================] - 13s 114ms/step - loss: 0.0079 - accuracy: 0.9909
Epoch 2/3
111/111 [==============================] - 12s 111ms/step - loss: 5.1331e-09 - accuracy: 1.0000
Epoch 3/3
111/111 [==============================] - 13s 114ms/step - loss: 2.8029e-09 - accuracy: 1.0000
Training for fold 2 ...
Found 801 images belonging to 2 classes.
Epoch 1/3
26/26 [==============================] - 4s 132ms/step - loss: 0.0216 - accuracy: 1.0000
Epoch 2/3
26/26 [==============================] - 3s 100ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 3/3
26/26 [==============================] - 3s 98ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Testing configuration: ((5, 5), (2, 2), (3, 3), (2, 2))
Training for fold 3 ...
Found 3530 images belonging to 2 classes.
Epoch 1/3
111/111 [==============================] - 8s 68ms/step - loss:

This code is designed to train a Convolutional Neural Network (CNN) model for image classification with different configurations and data augmentation. It uses the Keras library to define the CNN model and the ImageDataGenerator class for real-time data augmentation. The code tests different configurations of kernel size, stride for convolutional layers, and pooling size, stride for pooling layers. For each configuration, the model is trained on images from different directories and the accuracy is evaluated. The configuration yielding the highest accuracy is considered the best, and the corresponding model is saved. The code prints the configuration being tested and, at the end, the best configuration. Data augmentation techniques used include rotation, width and height shifts, shear transformation, zooming, and horizontal flipping.

# part 5

In [9]:
from keras.applications import VGG19, ResNet50
from keras.models import Sequential
from keras.layers import Dense, Flatten
from keras.preprocessing.image import ImageDataGenerator

# Load pre-trained models
vgg19 = VGG19(weights='imagenet', include_top=False, input_shape=(64, 64, 3))
resnet50 = ResNet50(weights='imagenet', include_top=False, input_shape=(64, 64, 3))

# Freeze the layers
for layer in vgg19.layers:
    layer.trainable = False

for layer in resnet50.layers:
    layer.trainable = False

# Create new models for transfer learning
model_vgg19 = Sequential([
    vgg19,
    Flatten(),
    Dense(units=128, activation='relu'),
    Dense(units=2, activation='softmax')
])

model_resnet50 = Sequential([
    resnet50,
    Flatten(),
    Dense(units=128, activation='relu'),
    Dense(units=2, activation='softmax')
])

# Compile the models
model_vgg19.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model_resnet50.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Prepare the image data generator
datagen = ImageDataGenerator(rescale=1./255)

# Define the directory
directory = '/content/train'

# Prepare the training data
train_data = datagen.flow_from_directory(
    directory,
    target_size=(64, 64),
    batch_size=32,
    class_mode='categorical'
)

# Train the models
model_vgg19.fit(train_data, epochs=3, verbose=1)
model_resnet50.fit(train_data, epochs=3, verbose=1)


Found 3530 images belonging to 2 classes.
Epoch 1/3
111/111 [==============================] - 138s 1s/step - loss: 0.0083 - accuracy: 0.9932
Epoch 2/3
111/111 [==============================] - 137s 1s/step - loss: 1.8911e-09 - accuracy: 1.0000
Epoch 3/3
111/111 [==============================] - 136s 1s/step - loss: 1.8911e-09 - accuracy: 1.0000
Epoch 1/3
111/111 [==============================] - 36s 302ms/step - loss: 0.0040 - accuracy: 1.0000
Epoch 2/3
111/111 [==============================] - 33s 300ms/step - loss: 2.5328e-08 - accuracy: 1.0000
Epoch 3/3
111/111 [==============================] - 34s 300ms/step - loss: 3.2420e-09 - accuracy: 1.0000


This code performs transfer learning using the VGG19 and ResNet50 models from Keras applications for image classification. The pre-trained models are loaded with weights trained on ImageNet, and their layers are frozen to preserve the learned features. New models are created by adding a flatten layer and two dense layers to the pre-trained models. The models are compiled and trained on training data prepared using Keras’s ImageDataGenerator. The code is designed to train the models on images from a specified directory and print the training progress.

# part 7

The size of the receptive field in convolutional layers, often referred to as the kernel size, can significantly impact the performance of a neural network.

When the kernel size is increased, the model becomes capable of capturing more global information in the input. Larger kernels can detect larger and more complex patterns in the image. However, this comes at the cost of losing some local details and requires more computational resources due to the increase in the number of parameters.

On the other hand, decreasing the kernel size allows the model to capture more local and detailed information in the input. Smaller kernels can detect smaller and simpler patterns in the image. This can be beneficial for tasks that require fine-grained understanding of the image. However, it might fail to capture larger patterns and the overall structure of the image.

Therefore, the choice of kernel size is a trade-off between capturing global structural information and local detailed information, and should be chosen based on the specific task and dataset. It’s also common to use a combination of different kernel sizes in the same model to capture features at various scales.

It is important to note that these are general observations and the actual impact may vary depending on the specific task, dataset, and overall architecture of the neural network. It’s always a good idea to experiment with different kernel sizes and observe their impact on the model’s performance.